In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# # Drop the null rows
df = df.dropna()
df.koi_disposition.value_counts()

FALSE POSITIVE    3504
CONFIRMED         1800
CANDIDATE         1687
Name: koi_disposition, dtype: int64

In [3]:
df.koi_disposition[df.koi_disposition == 'CANDIDATE'] = 1
df.koi_disposition[df.koi_disposition == 'FALSE POSITIVE'] = 0
df.koi_disposition[df.koi_disposition == 'CONFIRMED'] = 0
df2 = df.koi_disposition.astype(float)
df2
target_names  = ["Candidate", "Not"]

/Users/pest/anaconda3/envs/PythonData=3.6/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/pest/anaconda3/envs/PythonData=3.6/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/pest/anaconda3/envs/PythonData=3.6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

In [4]:
data = df.drop("koi_disposition", axis=1)
target = df2.values.reshape(-1,1)
print(data.shape, target.shape)
target.dtype

(6991, 40) (6991, 1)


dtype('float64')

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42, stratify=target)
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
4222,0,0,1,0,15.745559,0.000122,-0.000122,145.541110,0.006210,-0.006210,...,-199,4.495,0.054,-0.229,0.950,0.324,-0.101,293.67361,42.666458,13.276
6119,0,0,1,1,1.891144,0.000033,-0.000033,132.754500,0.016400,-0.016400,...,-128,4.637,0.049,-0.021,0.627,0.045,-0.055,290.29175,37.981079,13.814
3996,0,1,0,0,2.900613,0.000001,-0.000001,133.422871,0.000465,-0.000465,...,-209,4.472,0.081,-0.189,0.931,0.277,-0.119,296.43033,39.443489,16.943
374,0,0,0,0,3.481592,0.000010,-0.000010,132.988280,0.002320,-0.002320,...,-97,4.374,0.121,-0.088,1.022,0.129,-0.129,290.68225,37.355110,13.615
984,0,0,0,0,17.815969,0.000090,-0.000090,134.483200,0.004090,-0.004090,...,-131,4.484,0.040,-0.120,0.933,0.136,-0.063,293.98157,46.687752,15.200


In [7]:
X_minmax = MinMaxScaler().fit(X_train)
y_minmax = MinMaxScaler().fit(y_train)

X_train_minmax = X_minmax.transform(X_train)
X_test_minmax = X_minmax.transform(X_test)
y_train_minmax = y_minmax.transform(y_train)
y_test_minmax = y_minmax.transform(y_test)

In [8]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train_minmax, y_train_minmax)
predictions = model.predict(X_test_minmax)

/Users/pest/anaconda3/envs/PythonData=3.6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
print(f"Training Data Score: {model.score(X_train_minmax, y_train_minmax)}")
print(f"Testing Data Score: {model.score(X_test_minmax, y_test_minmax)}")

Training Data Score: 0.8495136372305931
Testing Data Score: 0.8398169336384439


In [15]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [200, 500, 1000],
              'gamma': [0.00001, 0.0001, 0.001]}
grid = GridSearchCV(model, param_grid, verbose=5)

In [16]:
grid.fit(X_train_minmax, y_train_minmax)

/Users/pest/anaconda3/envs/PythonData=3.6/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/pest/anaconda3/envs/PythonData=3.6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] C=200, gamma=1e-05 ..............................................
[CV] .................. C=200, gamma=1e-05, score=0.890, total=   0.6s
[CV] C=200, gamma=1e-05 ..............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
/Users/pest/anaconda3/envs/PythonData=3.6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] .................. C=200, gamma=1e-05, score=0.893, total=   0.5s
[CV] C=200, gamma=1e-05 ..............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.1s remaining:    0.0s
/Users/pest/anaconda3/envs/PythonData=3.6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

[CV] ................. C=1000, gamma=0.001, score=0.896, total=   1.1s
[CV] C=1000, gamma=0.001 .............................................


/Users/pest/anaconda3/envs/PythonData=3.6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV] ................. C=1000, gamma=0.001, score=0.879, total=   1.1s


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:   21.0s finished
/Users/pest/anaconda3/envs/PythonData=3.6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='linear',
                           max_iter=-1, probability=False, random_state=None,
                           shrinking=True, tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [200, 500, 1000],
                         'gamma': [1e-05, 0.0001, 0.001]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=5)

In [21]:
print(grid.best_params_)
print({grid.best_score_})

{'C': 500, 'gamma': 1e-05}
{0.8897577722677856}


In [18]:
from sklearn.metrics import classification_report
predictions = grid.predict(X_test_minmax)
print(classification_report(y_test_minmax, predictions,
                            target_names=target_names))

              precision    recall  f1-score   support

   Candidate       0.90      0.95      0.93      1326
         Not       0.81      0.68      0.74       422

    accuracy                           0.88      1748
   macro avg       0.86      0.82      0.83      1748
weighted avg       0.88      0.88      0.88      1748



In [14]:
import joblib